# Flight Delays Data Analysis - Initial Exploration

This notebook contains the initial exploration of flight delays data from both CSV files and API sources. We'll go through the process of loading, cleaning, and performing basic analysis of the data.

## Import Required Libraries

First, let's import all the necessary libraries for our analysis.

In [2]:
# Data manipulation and analysis
import pandas as pd
import duckdb

# API requests
import requests

# Utilities
from pathlib import Path
from tqdm.auto import tqdm

## Data Loading
This section will cover methods to import the flight delay data from CSVs into a pandas dataframe. First, we being by creating some constant variables for easy file path referencing.

In [ ]:
# Define directory paths
DATA_DIR = Path.cwd().parent.parent / "data"
RAW_DATA_DIR = DATA_DIR / "raw"
PROCESSED_DATA_DIR = DATA_DIR / "processed"

In [4]:
# Create list of file paths for each CSV
file_paths = [file_name for file_name in sorted(RAW_DATA_DIR.iterdir()) if ".csv" == file_name.suffix]

### Loading CSVs Directly & Then Filtering
This was my first naive approach to loading and filtering the flight delay data. **This method will not work in a google colab notebook because loading each dataset fully into memory eats up too much RAM**.

In [5]:
%%time

# Create an empty df, iterate through paths, load data into df, 
# filter df for PHL, concat to originally empty df and overwrite.
data_raw = pd.DataFrame()
for path in file_paths:
    print(f"----- Appending {path.name} -----")
    
    data_new = pd.read_csv(path)
    print(f"{path.name} full data memory usage: {data_new.memory_usage(deep = True).sum()} bytes.")
    data_filtered = data_new[data_new["ORIGIN"] == "PHL"]
    print(f"Filtered {path.name} data memory usage: {data_filtered.memory_usage(deep = True).sum()} bytes.")
    data_raw = pd.concat([data_raw, data_filtered])
    print(f"Raw data after {path.name} memory usage: {data_raw.memory_usage(deep = True).sum()} bytes.")
    print("")

    
print("Done!")

----- Appending 2009.csv -----
2009.csv full data memory usage: 2766182156 bytes.
Filtered 2009.csv data memory usage: 40697706 bytes.
Raw data after 2009.csv memory usage: 40697706 bytes.

----- Appending 2010.csv -----
2010.csv full data memory usage: 2775589032 bytes.
Filtered 2010.csv data memory usage: 38854980 bytes.
Raw data after 2010.csv memory usage: 79552686 bytes.

----- Appending 2011.csv -----
2011.csv full data memory usage: 2610748280 bytes.
Filtered 2011.csv data memory usage: 37716360 bytes.
Raw data after 2011.csv memory usage: 117269046 bytes.

----- Appending 2012.csv -----
2012.csv full data memory usage: 2623027308 bytes.
Filtered 2012.csv data memory usage: 34413126 bytes.
Raw data after 2012.csv memory usage: 151682172 bytes.

----- Appending 2013.csv -----
2013.csv full data memory usage: 2740605608 bytes.
Filtered 2013.csv data memory usage: 35495742 bytes.
Raw data after 2013.csv memory usage: 187177914 bytes.

Done!
CPU times: user 31.1 s, sys: 3.89 s, tota

### Load CSVs With Filtered Streaming
This approach worked inside of Google Colab. With this method, the only data loaded into memory is a 100,000 row chunk of the CSV. This chunk is then filtered before being appended to a list. This prevents us from loading the entire CSV into memory.

In [6]:
%%time

# Chunk each CSV, filter the chunk and load into memory,
# append to a list, concatenate list of filtered chunks together
data_to_concat = []

for path in (pbar := tqdm(file_paths)):
    pbar.set_description(f"Processing & Adding {path.name}")
    chunks = pd.read_csv(path, chunksize=100_000, low_memory = False)
    
    # For each chunk, filter for just PHL, add to concat list
    chunked_total = 0
    
    for chunk in tqdm(chunks, desc = f"Processing chunks from {path}", leave = False):
        # This is where chunk is loaded into memory
        chunk_filtered = chunk[chunk["ORIGIN"] == "PHL"]
        chunked_total += chunk_filtered.memory_usage(deep = True).sum()
        
        if not chunk_filtered.empty:
            data_to_concat.append(chunk_filtered)
    
    print(f"{path.name} chunked & filtered data memory usage: {chunked_total} bytes.")

print("")
data_raw = pd.concat(data_to_concat, ignore_index=True)
print(f"Raw data total memory usage: {data_raw.memory_usage(deep = True).sum()} bytes.")
print("")

  0%|          | 0/5 [00:00<?, ?it/s]

Processing chunks from /Users/rohankrishnan/Documents/GitHub/flight-delays-prediction/data/raw/2009.csv: 0it […

2009.csv chunked & filtered data memory usage: 40697706 bytes.


Processing chunks from /Users/rohankrishnan/Documents/GitHub/flight-delays-prediction/data/raw/2010.csv: 0it […

2010.csv chunked & filtered data memory usage: 38854980 bytes.


...

2011.csv chunked & filtered data memory usage: 37716360 bytes.


Processing chunks from /Users/rohankrishnan/Documents/GitHub/flight-delays-prediction/data/raw/2012.csv: 0it […

2012.csv chunked & filtered data memory usage: 34413126 bytes.


Processing chunks from /Users/rohankrishnan/Documents/GitHub/flight-delays-prediction/data/raw/2013.csv: 0it […

2013.csv chunked & filtered data memory usage: 35495742 bytes.

Raw data total memory usage: 183761910 bytes.

CPU times: user 25.1 s, sys: 3.32 s, total: 28.4 s
Wall time: 28.5 s


### Load CSVs With DuckDB
This approach should be the fastest. DuckDB allows us to query the data before loading it into memory so we only load the filtered data. I did this one last because in order to use `read_csv_auto` without error, we need to pass a schema to teach DuckDB the dtypes for each column.

In [17]:
%%time

# Create schema dict using dtype_map to translate pandas dtypes to SQL dtypes
# If we were to just use duckdb from the get-go, this would probably need to be
# made by hand
schema = {}
dtype_map = {
    "object": "VARCHAR",
    "int64": "BIGINT",
    "float64": "DOUBLE",
    "bool": "BOOLEAN",
    "datetime64[ns]": "TIMESTAMP"
}
for column in data_raw.columns:
    schema[column] = dtype_map[str(data_raw[column].dtype)]

# DuckDB connection, make data view, query data and store as df
conn = duckdb.connect()

data_view = duckdb.read_csv(
    f"{RAW_DATA_DIR}/*.csv",
    columns = schema,
    auto_detect = True
)

data_raw = duckdb.sql("""
    SELECT *
    FROM data_view
    WHERE DEST = 'PHL';
"""   
).df()

print(f"DuckDB raw data total memory usage: {data_raw.memory_usage(deep = True).sum()} bytes.")
print("")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

DuckDB raw data total memory usage: 180392108 bytes.

CPU times: user 26.2 s, sys: 991 ms, total: 27.2 s
Wall time: 3.27 s


In [8]:
data_raw.head(5)

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
0,2009-01-01,XE,2248,PHL,CLE,1312.0,1311.0,-1.0,9.0,1320.0,...,92.0,78.0,64.0,363.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2009-01-01,XE,2545,PHL,CLE,1750.0,1801.0,11.0,18.0,1819.0,...,98.0,87.0,62.0,363.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2009-01-01,YV,7144,PHL,IAD,1010.0,958.0,-12.0,17.0,1015.0,...,83.0,57.0,34.0,134.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2009-01-01,YV,7218,PHL,IAD,1910.0,1910.0,0.0,12.0,1922.0,...,69.0,52.0,32.0,134.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2009-01-01,NW,1761,PHL,DTW,2000.0,1949.0,-11.0,14.0,2003.0,...,120.0,95.0,77.0,453.0,NaN,NaN,NaN,NaN,NaN,NaN


# Weather Data API Call
This section covers how to call the OpenMeteo API and construct a dataframe of weather data.

In [18]:
# Define variables using the data & API Website variable selector
date_start, date_end = min(pd.to_datetime(data_raw["FL_DATE"])), max(pd.to_datetime(data_raw["FL_DATE"]))

daily_vars = [
    "temperature_2m_mean",
    "temperature_2m_max",
    "temperature_2m_min",
    "apparent_temperature_mean",
    "apparent_temperature_max",
    "apparent_temperature_min",
    "wind_speed_10m_max",
    "wind_gusts_10m_max",
    "wind_direction_10m_dominant",
    "shortwave_radiation_sum",
    "et0_fao_evapotranspiration",
    "precipitation_sum",
    "rain_sum",
    "snowfall_sum",
    "precipitation_hours",
    "weather_code",
]

In [19]:
# Call API with params
API_URL = "https://archive-api.open-meteo.com/v1/archive"


params = {
    "latitude": 38.87, # Googled
    "longitude": 75.24, # Googled
    "start_date": date_start.strftime("%Y-%m-%d"),
    "end_date": date_end.strftime("%Y-%m-%d"),
    "daily": daily_vars
}

response = requests.get(API_URL, params = params)
print(response.status_code) # Check that code is 200

200


In [20]:
# Extract daily weather data from json response and store as a pandas df
data_weather = pd.DataFrame(response.json()["daily"])

# Joining
This section covers joining the raw delay data with the weather data pulled from the OpenMeteo API.

In [21]:
# Make copies of each df before making edits
delays_df = data_raw.copy()
weather_df = data_weather.copy()

# Standardize date col types
delays_df["FL_DATE"] = pd.to_datetime(delays_df["FL_DATE"])
weather_df["time"] = pd.to_datetime(weather_df["time"])

In [22]:
# Left join delays_df with weather_df on respective date columns
combined_df = pd.merge(left = delays_df, right = weather_df,
                     how = "left", left_on = "FL_DATE", 
                     right_on = "time")

In [23]:
combined_df.head(5)

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,wind_speed_10m_max,wind_gusts_10m_max,wind_direction_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,weather_code
0,2009-01-01,XE,2514,CLE,PHL,1115.0,1108.0,-7.0,12.0,1120.0,...,8.7,45.7,225,10.84,0.52,0.0,0.0,0.0,0.0,3
1,2009-01-01,XE,2538,CLE,PHL,1545.0,1542.0,-3.0,13.0,1555.0,...,8.7,45.7,225,10.84,0.52,0.0,0.0,0.0,0.0,3
2,2009-01-01,YV,7144,IAD,PHL,832.0,833.0,1.0,14.0,847.0,...,8.7,45.7,225,10.84,0.52,0.0,0.0,0.0,0.0,3
3,2009-01-01,YV,7218,IAD,PHL,1710.0,1706.0,-4.0,17.0,1723.0,...,8.7,45.7,225,10.84,0.52,0.0,0.0,0.0,0.0,3
4,2009-01-01,NW,1762,DTW,PHL,1910.0,1905.0,-5.0,21.0,1926.0,...,8.7,45.7,225,10.84,0.52,0.0,0.0,0.0,0.0,3


The code and text above walks through the steps to load and combine the data. Next steps would be cleaning the data, engineering other features, and/or gathering other datasets to include in our analysis.

In [24]:
combined_df.to_csv("../data/processed/delay_weather_data.csv")

In [25]:
airport_codes = pd.read_csv("../data/raw/airports.csv")

In [26]:
airport_codes.head()

,code,icao,name,latitude,longitude,elevation,url,time_zone,city_code,country,city,state,county,type
0,AAA,NTGA,Anaa,-17.350665,-145.511120,36,NaN,Pacific/Tahiti,AAA,PF,NaN,NaN,NaN,AP
1,AAB,YARY,Arrabury Airport,-26.696783,141.049092,328,NaN,Australia/Brisbane,AAB,AU,Tanbar,Queensland,Barcoo Shire,AP
2,AAC,HEAR,El Arish International Airport,31.074284,33.829172,85,NaN,Africa/Cairo,AAC,EG,Arish,Muhafazat Shamal Sina',NaN,AP
3,AAD,HCAD,Adado Airport,6.096286,46.637708,980,NaN,Africa/Khartoum,AAD,SO,Adado,NaN,NaN,AP
4,AAE,DABB,Les Salines Airport,36.821392,7.811857,36,NaN,Africa/Algiers,AAE,DZ,El Hadjar,Annaba,NaN,AP


In [ ]:
airport_codes[airport_codes["country"] == "US"]["code"].unique()

array(['AAF', 'ABE', 'ABI', ..., 'ZNC', 'ZPH', 'ZZV'],
      shape=(2315,), dtype=object)

In [38]:
codes = list(combined_df["ORIGIN"].unique())

In [41]:
len(codes)

60

In [42]:
locations_df = airport_codes[airport_codes["code"].isin(codes)][["code", "latitude", "longitude"]]

In [43]:
locations_df

,code,latitude,longitude
237,ALB,42.745752,-73.809209
289,ANC,61.181035,-149.997892
429,ATL,33.637799,-84.429271
462,AUS,30.193489,-97.665010
638,BDL,41.940584,-72.685020
884,BNA,36.119590,-86.683087
927,BOS,42.362974,-71.013507
1043,BTV,44.472896,-73.151463
1052,BUF,42.940202,-78.730752
1103,BWI,39.179527,-76.668940


In [44]:
joined_df = pd.merge(left = combined_df, right = locations_df,
                     how = "left", left_on = "ORIGIN", right_on="code")

In [45]:
joined_df

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,shortwave_radiation_sum,et0_fao_evapotranspiration,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,weather_code,code,latitude,longitude
0,2009-01-01,XE,2514,CLE,PHL,1115.0,1108.0,-7.0,12.0,1120.0,...,10.84,0.52,0.0,0.0,0.0,0.0,3,CLE,41.406619,-81.851202
1,2009-01-01,XE,2538,CLE,PHL,1545.0,1542.0,-3.0,13.0,1555.0,...,10.84,0.52,0.0,0.0,0.0,0.0,3,CLE,41.406619,-81.851202
2,2009-01-01,YV,7144,IAD,PHL,832.0,833.0,1.0,14.0,847.0,...,10.84,0.52,0.0,0.0,0.0,0.0,3,IAD,38.952266,-77.453485
3,2009-01-01,YV,7218,IAD,PHL,1710.0,1706.0,-4.0,17.0,1723.0,...,10.84,0.52,0.0,0.0,0.0,0.0,3,IAD,38.952266,-77.453485
4,2009-01-01,NW,1762,DTW,PHL,1910.0,1905.0,-5.0,21.0,1926.0,...,10.84,0.52,0.0,0.0,0.0,0.0,3,DTW,42.205699,-83.352975
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426971,2013-12-31,FL,209,ATL,PHL,830.0,830.0,0.0,12.0,842.0,...,9.17,0.50,0.0,0.0,0.0,0.0,3,ATL,33.637799,-84.429271
426972,2013-12-31,FL,232,ATL,PHL,2100.0,2052.0,-8.0,11.0,2103.0,...,9.17,0.50,0.0,0.0,0.0,0.0,3,ATL,33.637799,-84.429271
426973,2013-12-31,FL,1022,MCO,PHL,2135.0,2152.0,17.0,10.0,2202.0,...,9.17,0.50,0.0,0.0,0.0,0.0,3,MCO,28.412904,-81.309443
426974,2013-12-31,MQ,3114,ORD,PHL,1105.0,1106.0,1.0,12.0,1118.0,...,9.17,0.50,0.0,0.0,0.0,0.0,3,ORD,41.977957,-87.909176


In [46]:
joined_df.columns

Index(['FL_DATE', 'OP_CARRIER', 'OP_CARRIER_FL_NUM', 'ORIGIN', 'DEST',
       'CRS_DEP_TIME', 'DEP_TIME', 'DEP_DELAY', 'TAXI_OUT', 'WHEELS_OFF',
       'WHEELS_ON', 'TAXI_IN', 'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY',
       'CANCELLED', 'CANCELLATION_CODE', 'DIVERTED', 'CRS_ELAPSED_TIME',
       'ACTUAL_ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'CARRIER_DELAY',
       'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY',
       'Unnamed: 27', 'time', 'temperature_2m_mean', 'temperature_2m_max',
       'temperature_2m_min', 'apparent_temperature_mean',
       'apparent_temperature_max', 'apparent_temperature_min',
       'wind_speed_10m_max', 'wind_gusts_10m_max',
       'wind_direction_10m_dominant', 'shortwave_radiation_sum',
       'et0_fao_evapotranspiration', 'precipitation_sum', 'rain_sum',
       'snowfall_sum', 'precipitation_hours', 'weather_code', 'code',
       'latitude', 'longitude'],
      dtype='object')

In [55]:
import pandas as pd
import plotly.graph_objects as go

# df: flights where DEST == 'PHL'
# Example:
# df = full_df[full_df['DEST'] == 'PHL']

# 1. Compute avg arrival delay per origin airport
avg_delay = (
    joined_df.groupby('ORIGIN', as_index=False)
      .agg(avg_arr_delay=('ARR_DELAY', 'mean'),
           num_flights=('ARR_DELAY', 'count'),
           origin_lat=('latitude', 'first'),
           origin_lon=('longitude', 'first'))
)

# 2. Get PHL coordinates
phl_lat = 39.8730
phl_lon = -75.2437

# 3. Create figure
fig = go.Figure()

# 4. Add route lines (from each origin → PHL)
# for _, row in avg_delay.iterrows():
#     fig.add_trace(
#         go.Scattergeo(
#             lon=[row['origin_lon'], phl_lon],
#             lat=[row['origin_lat'], phl_lat],
#             mode='lines',
#             line=dict(
#                 width=max(row['avg_arr_delay'] / 10, 1),
#                 color='red' if row['avg_arr_delay'] > 15 else 'green'
#             ),
#             opacity=0.6,
#             hoverinfo='text',
#             text=f"{row['ORIGIN']} → PHL<br>"
#                  f"Avg Delay: {row['avg_arr_delay']:.1f} min<br>"
#                  f"Flights: {row['num_flights']}"
#         )
#     )

# 5. Add origin airport markers (with hover info)
fig.add_trace(
    go.Scattergeo(
        lon=avg_delay['origin_lon'],
        lat=avg_delay['origin_lat'],
        mode='markers',
        marker=dict(
            size=12,
            color=avg_delay['avg_arr_delay'],
            colorscale='RdYlGn_r',  # red = high delay, green = low delay
            colorbar=dict(title="Avg Delay (min)"),
            showscale=True,
            opacity=0.9
        ),
        hoverinfo='text',
        text=(
            "Origin: " + avg_delay['ORIGIN'] +
            "<br>Avg Delay: " + avg_delay['avg_arr_delay'].round(1).astype(str) + " min" +
            "<br>Flights: " + avg_delay['num_flights'].astype(str)
        ),
        name='Origin Airports'
    )
)

# 6. Add PHL marker
fig.add_trace(
    go.Scattergeo(
        lon=[phl_lon],
        lat=[phl_lat],
        text='PHL (Destination)',
        mode='markers+text',
        textposition='top center',
        marker=dict(size=10, color='orange', symbol='star'),
        name='PHL'
    )
)

# 7. Layout
fig.update_layout(
    title="Average Arrival Delays for Flights into Philadelphia (PHL)",
    geo=dict(
        scope='north america',
        projection_type='azimuthal equal area',
        showland=True,
        landcolor="rgb(243, 243, 243)",
        countrycolor="rgb(204, 204, 204)",
    ),
    height=700
)

fig.show()


In [56]:
joined_df.to_csv("../data/processed/delay_weather_coord.csv")